In [1]:
import requests
import re
import time
import lxml.html
import openpyxl
import pandas as pd

from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
custom_header = {
    'referer' : 'http://www.riss.kr/index.do',
    #'referer' : 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=&onHanja=&strSort=&p_year1=&p_year2=&iStartCount=0&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }

# 검색키워드: 청소년

In [3]:
#http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%B2%AD%EC%86%8C%EB%85%84&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EC%B2%AD%EC%86%8C%EB%85%84

In [4]:
paper_link_list = []
#49
for i in range(49):
    before_url = "http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%B2%AD%EC%86%8C%EB%85%84&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EC%B2%AD%EC%86%8C%EB%85%84"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

청소년용 정신화 프로그램의 학생정신건강 증진효과 평가
소아청소년 양극성 장애의 임상 경과
소아청소년 우울증에서의 환경적 위험 인자들과 임상적 의미
청소년정서행동발달검사 표준화연구 자료를 활용한 학교폭력 피해 전국유병률 및 관련요인 조사
소아청소년 집단에서 집단 따돌림의 유병률과 이와 관련된 정신병리 현상
1, 2호 보호 처분 청소년의 정신의학적 특성 및 6개월 보호처분 후 추적조사
단축형 청소년용 정신화 프로그램의 학생정신건강 증진효과 평가
청소년용 정신건강 선별검사 개정 연구
청소년의 휴대 전화 사용 실태 및 정신과적 문제와의 연관성
자아탄력-자아통제 수준에 따른 청소년의 기질 및 성격 특성
10
국내 주의력결핍 과잉행동장애 아동 및 청소년에서 메타데이트CD의 임상증상 및 신경인지기능 개선 효과에 대한 개방 연구
소아청소년 양극성장애 입원 환자의 약물 사용 특성 : 후향적 차트 분석 연구
청소년정서행동발달검사 표준화 연구 자료를 활용한 교사의 학교폭력 피해자 인지도
소아청소년 자폐 스펙트럼장애에서 메칠페니데이트 사용의 임상적 특징에 대한 후향적 분석
담배광고 목격경험과 담배회사의 사회공헌활동에 대한 선호도가 청소년의 현재 흡연과 미래 흡연의도에 미치는 영향
청소년 간접흡연 예방 프로그램의 개발 및 효과
청소년의 자살생각 경로모형 분석
생활권 청소년 수련시설과 교과교실제 학교시설의 복합화를 위한 기초연구
비행청소년의 감각처리능력과 문제행동에 관한 연구
초기 청소년의 감각처리특성 및 인성간의 관련성 연구
20
아동 청소년기 우울증의 예방 개입에 대한 고찰
쉼터 거주 청소년의 또래관계 향상과 우울감 감소를 위한 집단미술치료 사례연구
청소년 유해 매체물에 대한 방송사 자율규제에 관한 연구 : 자율심의제 정착방안을 중심으로
부모상담과 미술활동을 활용한 통합적 심리치료가 중년여성의 결혼만족도와 우울 및 청소년기 자녀의 공격성에 미치는 효과
집단미술치료가 청소년의 우울과 자살생각에 미치는 효과
통합예술치료를 활용한 사회적 역량 증진 프로그램이 보호관찰청소년에게 미치는 효

국내 청소년의 인플루엔자 예방접종률에 영향을 미치는 요인: 국민건강영양조사 4-6기 자료분석
아동·청소년의 대사증후군 및 대사이상 지표의 분포와 영양소 섭취
초기 청소년의 학년에 따른 개인적 변인, 사회적 지지와 자아 존중감 간의 인과적 관련성
한국 청소년의 고혈압과 관련된 식사 및 생활양식요인 분석
한국 청소년의 수면시간, 스트레스 및 우울증이 구강건강 상태에 미치는 영향
여자 청소년에서 식품군 점수에 따른 영양소 섭취 상태 및 비만 관련 신체계측치의 평가
한국 청소년에서 치아우식증 고위험군의 우식양상
지적장애 청소년의 장기간 육상운동이 기초체력과 신경전달물질 및 스트레스 호르몬에 미치는 영향
남녀 청소년 소비자의 온라인 문제행동 차이에 대한 종단 분석
일부 청소년의 흡연 실태와 흡연 관련 구강건강지식수준에 관한 연구
220
아동,청소년을 위한 자가평가용 자아개념 척도 개발
우리나라 청소년의 우유,유제품 섭취와 영양소 섭취 수준과의 관계
북한이탈 청소년의 문제행동과 문화변용정체성에 관한 연구
수영운동이 청소년기 흰쥐 뇌에서 BDNF 발현에 미치는 영향
전환기 청소년의 개인특성, 또래애착 및 부모양육행동이학교생활 적응에 미치는 영향: 휴대전화 의존도의 매개효과 검증
초기 청소년의 가출 충동에 영향을 미치는 요인
소아,청소년 정신병동 환아의 비만관리 프로그램 적용 효과
한국 청소년의 비만과 식습관 환경요인 변화 추이 분석
가족건강성이 청소년의 학교적응에 미치는 영향 및 사회적 지지의 매개효과 : 경남지역 중학생을 중심으로
다문화 가족 청소년의 체형지각, 건강행위 및 건강상태
230
부모 양육행동이 청소년의 감사성향에 미치는 영향
부모의 성취압력과 청소년의 자기불일치가 학업적 성취동기에 미치는 영향
일회성 농구운동이 청소년의 혈압과 혈관경직도에 미치는 영향
원저 : 청소년에서 메타졸아마이드와 아세타졸아마이드의 고산병 예방 효과 비교
연구논문 : 청소년의 스트레스에 따른 헤어스타일행동
부모의 행동통제와 청소년의 불안이 학교적응에 미치는 영향
연구논문 : 청소년의 뷰

원위 경골 및 비골에 발생한 만성 재발성 다발성 골수염
비만 관련 법령 현황 및 개선방향
연구논문(硏究論文) : 대중매체에 나타난 교복디자인 연구
중학교 학업성적에 따른 색채 선호도와 색채 사용수의 상관관계
돌연변이 육종에 의한 재래종 서리태 개량 신품종 콩 "조생서리"
국내 육가공품의 영양표시 현황과 소비자 인지도 및 소비경향 실태조사
행동활성화/억제체계가 폭식행동에 미치는 영향 : 정서조절곤란의 매개효과
악성 댓글에 대한 제3자 효과 연구
대학생의 감각처리유형과 자존감, 학습능력의 상관관계 연구
섬유매체를 활용한 집단미술치료가 중학생의 사회성에 미치는 효과
430
여고생의 문제행동양상과 문제행동 영향요인
초등학교 건강한 학교 만들기 사업에서 수행한 영양 및 신체상 교육 프로그램의 효과
아동기 정서·행동문제의 선별을 위한 인물화검사 타당화 연구
결정트리 데이터마이닝을 이용한 족부 임상 진단
완전탈구 후 재식한 영구 절치의 예후에 대한 후향적 연구
학령기 아동을 위한 지역사회 참여형 비만관리 프로그램의 효과
대학생의 수면과 감각처리 특성의 상관관계
충북지역 중학생의 식이섬유 섭취 실태 및 식품섭취빈도조사지 개발
초, 중, 고등학교 교과서의 안 보건 내용 현황
아동작업치료사의 직무분석 : 감각통합치료 중심으로
440
서울시 중학생들의 숲 체험 프로그램 만족도 및 요구도 분석
인문계·실업계 고등학생의 식사에 대한 인식과 영양지식 비교
스마트폰의 동영상과 게임 사용이 대학생의 안구건조에 미치는 영향
2015년 우리나라에서 발생한 중동호흡기증후군과 대한소아감염학회의 역할
난치성 소아 간질에서 levetiracetam의 효과
소아 간질 환자에서 항경련제 치료 후 간질파의 변화
한국인의 생애주기별 작업영역에 대한 시간사용
가와사키병 환자의 혈청 interleukin-6, tumor necrosis factor-α, Adiponectin 치에 대한 연구
신생아의 MRSA 균혈증에서 합병증 발생과 연관된 위험인자
극소 저체중 출생아에서 Synagis<sup>Ⓡ</sup> (

In [5]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=b84eee7adf877cceffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=88d6125c0aee9d38ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=af734a77095d156affe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=798c24527feb8f92ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=983b78292ab2a251ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e68328b36f6a1bd9ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=2670f13e33c5e651ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=a68fdcdce5c64501ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e6a800f0a3651c3effe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [6]:
len(paper_link_list)

481

In [7]:
paper_url = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url.append(tmp_url)

100%|████████████████████████████████████████████████████████████████████████████| 481/481 [00:00<00:00, 482530.55it/s]


In [8]:
paper_url

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=b84eee7adf877cceffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=88d6125c0aee9d38ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=af734a77095d156affe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=798c24527feb8f92ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=983b78292ab2a251ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e68328b36f6a1bd9ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=2670f13e33c5e651ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=a68fdcdce5c64501ffe0bdc3ef48d419',
 'http://www.ris

In [9]:
len(paper_url)

481

In [10]:
write_wb = openpyxl.Workbook()
write_ws = write_wb.active

In [11]:
"""
write_ws = write_wb.active
write_ws['A1'] = 'title_kor'
write_ws['B1'] = 'title_eng'
write_ws['C1'] = 'author'
write_ws['D1'] = 'book'
write_ws['E1'] = 'year'
write_ws['F1'] = 'keyword'
write_ws['G1'] = 'abstract_kor'
write_ws['H1'] = 'abstract_eng'
"""

"\nwrite_ws = write_wb.active\nwrite_ws['A1'] = 'title_kor'\nwrite_ws['B1'] = 'title_eng'\nwrite_ws['C1'] = 'author'\nwrite_ws['D1'] = 'book'\nwrite_ws['E1'] = 'year'\nwrite_ws['F1'] = 'keyword'\nwrite_ws['G1'] = 'abstract_kor'\nwrite_ws['H1'] = 'abstract_eng'\n"

In [12]:
write_ws = write_wb.active
write_ws['A1'] = 'title'
write_ws['B1'] = 'author'
write_ws['C1'] = 'book'
write_ws['D1'] = 'year'
write_ws['E1'] = 'keyword'
write_ws['F1'] = 'abstract_kor'
write_ws['G1'] = 'abstract_eng'

In [13]:
start = time.time()

for i, url in enumerate(paper_url):
    res = requests.get(url, headers = custom_header)
    element = lxml.html.fromstring(res.text)

    try:
        tmp_title = element.cssselect('h3.title')
        tmp_title = tmp_title[0].text_content()
        tmp_title = tmp_title.split("=")
        title_kor = tmp_title[0].strip()
        #title_eng = tmp_title[1].strip()

        tmp_author = element.cssselect('div.infoDetailL > ul > li:nth-child(1) > div > p')
        author = tmp_author[0].text_content().strip()

        tmp_book = element.cssselect('div.infoDetailL > ul > li:nth-child(3) > div > p')
        book = tmp_book[0].text_content().strip()

        tmp_year = element.cssselect('div.infoDetailL > ul > li:nth-child(5) > div > p')
        year = tmp_year[0].text_content().strip()

        tmp_keyword = element.cssselect('div.infoDetailL > ul > li:nth-child(7) > div > p')
        keyword = tmp_keyword[0].text_content().strip()

        tmp_abstract = element.cssselect('div.text.off')
        abstract_kor = tmp_abstract[0].text_content().strip()
        abstract_eng = tmp_abstract[1].text_content().strip()

        print(i, 'paper clear')

    except:
        print(i, 'ERROR')
        continue


    write_ws.append([title_kor, author, book, year, keyword, abstract_kor, abstract_eng])
    write_wb.save('./7_청소년.xlsx')
    
print("Running Time : ", time.time() - start)

0 paper clear
1 paper clear
2 paper clear
3 paper clear
4 paper clear
5 paper clear
6 paper clear
7 paper clear
8 paper clear
9 paper clear
10 paper clear
11 paper clear
12 paper clear
13 paper clear
14 paper clear
15 paper clear
16 paper clear
17 paper clear
18 paper clear
19 paper clear
20 paper clear
21 paper clear
22 paper clear
23 paper clear
24 paper clear
25 paper clear
26 paper clear
27 paper clear
28 paper clear
29 paper clear
30 paper clear
31 paper clear
32 paper clear
33 paper clear
34 paper clear
35 paper clear
36 paper clear
37 paper clear
38 paper clear
39 paper clear
40 paper clear
41 paper clear
42 paper clear
43 paper clear
44 paper clear
45 paper clear
46 paper clear
47 paper clear
48 paper clear
49 paper clear
50 paper clear
51 paper clear
52 paper clear
53 paper clear
54 paper clear
55 paper clear
56 paper clear
57 paper clear
58 paper clear
59 paper clear
60 paper clear
61 paper clear
62 paper clear
63 paper clear
64 paper clear
65 paper clear
66 paper clear
67 pa